### Generate query - reference context pairs:

In [1]:
import os
import sys
from pathlib import Path
import uuid
from tqdm import tqdm
from typing import List
import pandas as pd
import re
import json
from dotenv import load_dotenv
sys.path.append(str(Path.cwd().parent.parent))

from llama_index.core import Document 
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.schema import MetadataMode, TextNode
from qdrant_client import QdrantClient

load_dotenv(override=True)

True

In [2]:
# A customized prompts:
QUERY_REFERENCE_CONTEXTS_PAIRS_PROMPT = """\
Thông tin ngữ cảnh được cung cấp dưới đây.

---------------------
{context_str}
---------------------
  
Dựa vào thông tin trong phần context, không sử dụng kiến thức bên ngoài.

Bạn đang đóng vai là một chuyên viên tư vấn pháp luật, có nhiệm vụ hỗ trợ người dân Việt Nam hiểu rõ hơn về luật giao thông đường bộ.

Hãy đặt ra {num_questions_per_chunk} câu hỏi mà người dân có thể quan tâm hoặc thường đặt ra khi tiếp cận nội dung này.  
Câu hỏi cần rõ ràng, dễ hiểu, phản ánh thắc mắc thực tế của người dân về quy định, quyền lợi, nghĩa vụ hoặc cách áp dụng luật.

Yêu cầu về văn phong:
- Câu hỏi nên có giọng điệu kết hợp giữa ngôn ngữ đời thường (gần gũi, dễ tiếp cận) và cách diễn đạt chính xác, trang trọng khi cần.
- Ví dụ:  
  - "Nếu mình chở hàng nguy hiểm mà không có giấy phép thì bị sao vậy?"  
  - "Cá nhân vi phạm quy định về tốc độ tối đa sẽ bị xử phạt như thế nào theo điều luật này?"

Chỉ đặt câu hỏi dựa trên nội dung được cung cấp, không dùng kiến thức bên ngoài.
"""


In [3]:
def get_nodes_from_collection(collection_name: str):
    '''
    Get all nodes from a qdrant collection
    Args:
        collection_name: a qdrant collection name
    '''
    client = QdrantClient(url="http://localhost:6333")

    qdrant_nodes, _ = client.scroll(
        collection_name="contextual_rag_nckh",
        limit=1489
    )
    nodes = []
    for node in qdrant_nodes:
        nodes.append(TextNode(text=node.payload['text'], id_=node.id))
    return nodes

In [4]:
nodes = get_nodes_from_collection('contextual_rag_nckh')

/tmp/ipykernel_301632/1079936399.py:7: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.11.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  client = QdrantClient(url="http://localhost:6333")


In [24]:
def generate_query_ref_context_pairs(
    nodes: List[TextNode],
    prompt: str = QUERY_REFERENCE_CONTEXTS_PAIRS_PROMPT,
    num_question: int = 2,
    output_path: str = "query_ref_context_pairs.json"
) -> None:
    '''
    Generate query-reference contexts pairs with customized prompts.

    Args:
        nodes (List[TextNode]): list of LlamaIndex nodes
        prompt (str): instruction prompt that simulates user request
        num_question (int): number of questions per context
        output_path (str): file path to save the results in JSON
    '''

    llm = OpenAI(
        model='gpt-4o-mini',
        api_key=os.getenv('OPENAI_API_KEY')
    )
    Settings.llm = llm

    node_dict = {
        node.node_id: node.get_content(metadata_mode=MetadataMode.NONE)
        for node in nodes
    }

    queries = {}
    relevant_ids = {}
    relevant_texts = {}

    for node_id, text in tqdm(node_dict.items()):
        query_prompt = prompt.format(
            context_str=text,
            num_questions_per_chunk=num_question
        )
        response = llm.complete(query_prompt)
        result = str(response).strip().split("\n")
        questions = [re.sub(r"^\d+[\).\s]*", "", question).strip() for question in result if question.strip()]
        for question in questions:
            question_id = str(uuid.uuid4())
            queries[question_id] = question
            relevant_ids[question_id] = [node_id]
            relevant_texts[question_id] = [text]

    # Gộp lại và ghi ra file JSON
    output_data = [
        {
            "question_id": qid,
            "question": queries[qid],
            "relevant_node_ids": relevant_ids[qid],
            "relevant_node_text": relevant_texts[qid]
        }
        for qid in queries
    ]

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    print(f">>> Saved {len(output_data)} query-reference pairs to {output_path}")


In [25]:
generate_query_ref_context_pairs(nodes)

100%|██████████| 1489/1489 [49:00<00:00,  1.98s/it] 


>>> Saved 2978 query-reference pairs to query_ref_context_pairs.json


### MRR Metrics:

##### Custom retrieval:

In [5]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from llama_index.llms.openai import OpenAI
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import (
    Settings,
    Document,
    QueryBundle,
    StorageContext,
    VectorStoreIndex,
)

import nest_asyncio
nest_asyncio.apply()


In [6]:
# Init Êmbedding Model:
embed_model = OpenAIEmbedding(model='text-embedding-3-large', api_key=os.getenv('OPENAI_API_KEY'))
Settings.embed_model = embed_model
llm = OpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
Settings.llm = llm


# some  pre-init
qdrant_client = QdrantClient(
            url = 'http://localhost:6333'
        )

vector_store = QdrantVectorStore(client=qdrant_client, collection_name='contextual_rag_nckh')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
qdrant_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, 
    storage_context=storage_context,
    use_async=True)
    
retriever = VectorIndexRetriever(
            index=qdrant_index,
            similarity_top_k = 10,
            use_async = True
        )
# Initialize reranker
reranker_gpt = RankGPTRerank(
    llm=OpenAI(
        model="gpt-4o-mini",
        temperature=0.0,
        api_key=os.getenv('OPENAI_API_KEY'),
    ),
    top_n=3,
    verbose=True,
)

# Initialize retriever
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[reranker_gpt])

/tmp/ipykernel_301632/352016665.py:9: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.11.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  qdrant_client = QdrantClient(


In [7]:
def retrieve_top_k_documents(query):
    '''
    Search the query with the contextual RAG
    
    Args:
        query (str): The query string
        k (int): The number of documents to return  
    ''' 
    
    semantic_results = query_engine.query(query)

    return semantic_results.source_nodes


In [8]:
# Test Retrieval:
retrieve_top_k_documents('Nếu chủ đầu tư không chọn tổ chức thẩm tra an toàn giao thông đủ điều kiện, thì công trình đường bộ đó có được đưa vào khai thác không?')

After Reranking, new rank list for nodes: [0, 1, 4, 3, 2, 6, 9, 8, 7, 5]

[NodeWithScore(node=TextNode(id_='0014e974-14a9-4ec5-895b-a8cf57914cc4', embedding=None, metadata={'chapter_uuid': '', 'original_content': 'Trước khi đưa công trình đường bộ vào khai thác, chủ đầu tư có trách nhiệm: a) Lựa chọn tổ chức kinh doanh dịch vụ thẩm tra an toàn giao thông đủ điều kiện để thực hiện thẩm tra an toàn giao thông; b) Thực hiện thẩm định an toàn giao thông trên cơ sở báo cáo thẩm tra an toàn giao thông của tổ chức tư vấn thẩm tra an toàn giao thông.', 'article_uuid': '0014e974-14a9-4ec5-895b-a8cf57914cc4', 'khoan': 'khoản 2', 'dieu': 'Điều 32. Thẩm tra, thẩm định an toàn giao thông đường bộ', 'chuong': 'Chương V THẨM TRA, THẨM ĐỊNH AN TOÀN GIAO THÔNG ĐƯỜNG BỘ, ĐÀO TẠO THẨM TRA AN TOÀN GIAO THÔNG ĐƯỜNG BỘ', 'luat': 'Nghị định 165/2024/NĐ-CP hướng dẫn Luật Đường bộ và Điều 77 Luật Trật tự an toàn giao thông đường bộ'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Nội dung tại Điều 32 trong Nghị định 165/2024/NĐ-CP quy định v

##### MRR Metric Compute:

In [9]:
import json
from tqdm import tqdm

def compute_mrr_dataset(dataset_json_path: str, output_path: str = "mrr_results.json"):
    '''
    Compute Mean Reciprocal Rank for each retrieved node and save results
    Args:
        dataset_json_path: path to dataset
        output_path: path to save the detailed MRR results
    '''
    def get_node_uuid(node):
        return node.node.metadata['article_uuid']

    with open(dataset_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    reciprocal_ranks = []
    results = []

    for item in tqdm(data):
        question_id = item.get("question_id")
        question = item.get("question")
        relevant_node_ids = item.get("relevant_node_ids", [])

        retrieved_node_ids = [get_node_uuid(node) for node in retrieve_top_k_documents(question)]

        # Tính RR cho từng node
        node_rr_list = []
        for rank, node_id in enumerate(retrieved_node_ids, start=1):
            rr = 1.0 / rank if node_id in relevant_node_ids else 0.0
            node_rr_list.append({
                "node_id": node_id,
                "rank": rank,
                "reciprocal_rank": rr
            })

        # Tính RR chung cho câu hỏi (là RR đầu tiên > 0)
        first_hit = next((n for n in node_rr_list if n["reciprocal_rank"] > 0), None)
        overall_rr = first_hit["reciprocal_rank"] if first_hit else 0.0
        reciprocal_ranks.append(overall_rr)

        results.append({
            "question_id": question_id,
            "question": question,
            "relevant_node_ids": relevant_node_ids,
            "retrieved_node_rr": node_rr_list,
            "reciprocal_rank": overall_rr
        })

    # Mean Reciprocal Rank
    mrr = sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0.0

    # Save results
    with open(output_path, "w", encoding="utf-8") as out_f:
        json.dump({
            "mean_reciprocal_rank": mrr,
            "details": results
        }, out_f, ensure_ascii=False, indent=2)

    return mrr


In [ ]:
mrr = compute_mrr_dataset('/workspace/competitions/Sly/RAG_Traffic_Law_experiment/duy/notebook/evaluate/source/query_ref_context_pairs.json')

In [15]:
mrr